<a href="https://colab.research.google.com/github/jwoonge/ML-projects/blob/master/10/assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimal Selection of the hyper-parameters associated with the classification on MNIST
20141261 송제웅  
<hr>  

## 0. Import library
---
import library

In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import matplotlib.pyplot as plt

## 1. DownLoad and Normalize dataset (MNIST)
---  

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),
])

data_path = './MNIST'
data_train = datasets.MNIST(root = data_path, train = False, download = True, transform = transform)
data_test = datasets.MNIST(root = data_path, train = True, download = True, transform = transform)


## 2. Design NN Model
---
three fully connected layers with an activation function of sigmoid, LogSoftmax

In [3]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=64),
            #nn.Softsign(),
            #nn.Sigmoid()
            nn.ELU()
        )
        
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=64, out_features=48),
            #nn.Softsign(),
            #nn.Sigmoid()
            nn.ELU()
        )
        
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=48, out_features=10),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, inputs):
        x = inputs.view(inputs.size(0), -1)
        x = self.classifier1(x)
        x = self.classifier2(x)
        out = self.classifier3(x)
        return out

## 3. Define functions for learning
---

In [4]:
def train(model, data_train, data_train_batch, optimizer, criterion, device='cuda'):
    model.train()
    n_batch = 0
    avg_loss = 0
    avg_acc = 0
    for batch_idx, (x, y) in enumerate(data_train_batch):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model.forward(x)
        loss = criterion(pred, y)
        avg_loss += loss.item()
        avg_acc += accuracy(pred, y)
        n_batch += 1

        loss.backward()
        optimizer.step()

    avg_loss /= n_batch
    avg_acc /= n_batch
    return avg_loss, avg_acc

def test(model, x, y, criterion):
    model.eval()
    with torch.no_grad():
        pred = model.forward(x)
        loss = criterion(pred, y).item()
        acc = accuracy(pred, y)
    return loss, acc

def accuracy(pred, y):
    correct_cnt = 0
    num_sample = len(y)
    for i in range(num_sample):
        pred_class = pred[i].argmax()
        if y[i]==pred_class:
            correct_cnt += 1
    return 100 * correct_cnt / num_sample


In [11]:
def learn(model, data_train, data_test, criterion, optimizer, batch_size, epoch, device='cuda'):
    data_train_batch = torch.utils.data.DataLoader(data_train, batch_size, shuffle=True)
    test_x, test_y = data_test.test_data.view((60000,28*28)), data_test.test_labels
    test_x, test_y = torch.tensor(test_x, dtype=torch.float, device=device), test_y.to(device)
    #loss_train_s = []; loss_test_s = []; acc_train_s = []; acc_test_s = []
    i = 0
    while i<epoch:
        losstest, acctest = test(model, test_x, test_y, criterion)
        losstrain, acctrain = train(model, data_train, data_train_batch, optimizer, criterion, device)
        loss_train.append(losstrain); loss_test.append(losstest)
        acc_train.append(acctrain); acc_test.append(acctest)
        i += 1
        print(i,'\t', losstrain,'\t', acctrain,'\t', losstest,'\t', acctest)
        

In [6]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')


## 4. Learn
---

In [13]:
global loss_train, loss_test, acc_train, acc_test
loss_train = []; loss_test = []; acc_train = []; acc_test = []
classifier = classification().to(device)
optimizer = torch.optim.Adadelta(classifier.parameters(), lr=20, weight_decay=0.0001)
criterion = nn.NLLLoss()
learn(classifier, data_train, data_test, criterion, optimizer, 10000, 5, device)
optimizer = torch.optim.SGD(classifier.parameters(), lr=0.3, weight_decay=0.0005)
learn(classifier, data_train, data_test, criterion, optimizer, 10000, 5, device)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


1 	 2.34045147895813 	 8.32 	 11.430671691894531 	 11.488333333333333
2 	 3.3449699878692627 	 28.47 	 253.22015380859375 	 20.54
3 	 21.65400505065918 	 11.29 	 1194.865966796875 	 11.553333333333333
4 	 27.270252227783203 	 10.1 	 1534.3814697265625 	 10.255
5 	 18.187101364135742 	 17.37 	 820.8509521484375 	 18.661666666666665
1 	 18.01138687133789 	 8.92 	 634.569580078125 	 11.435
2 	 9.021909713745117 	 15.65 	 412.2015380859375 	 18.291666666666668
3 	 11.0753812789917 	 16.8 	 241.466552734375 	 23.98
4 	 3.461045742034912 	 29.5 	 207.7643280029297 	 24.586666666666666
5 	 5.405007362365723 	 28.56 	 185.74871826171875 	 27.321666666666665


---  
# RESULTS
---  
## 1. Plot the training and testing losses over epochs
---

In [ ]:
plt.plot(loss_train, color='r')
plt.plot(loss_test, color='b')
plt.title('loss')
plt.legend(['train','test'])
plt.show()

## 2. Plot the training and testing accuracies over epochs
---


In [ ]:
plt.plot(acc_train, color='r')
plt.plot(acc_test, color='b')
plt.title('accuracy(%)')
plt.legend(['train','test'])
plt.show()

## 3. Print the final training and testing losses at convergence
---

In [ ]:
print('training loss\t',loss_train[-1])
print('testing loss\t',loss_test[-1])

## 4. Print the final training and testing accuracies at convergence
---

In [ ]:
print('training accuracy\t',acc_train[-1])
print('testing accuracy\t',acc_test[-1])